# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [108]:
import json
import pickle
import pandas as pd
import requests
from bs4 import BeautifulSoup

with open ('C:/Users/Wiio/PycharmProjects/Makrushin_TOD/TOD_Homework_3/data/addres-book.json') as f:
    templates = json.load(f)
    


def json_selector(data: list, param: str) -> str:
    res = []
    for i in data:
        res.append(i[param])
    return res
        
print(*json_selector(templates, 'email'))

faina@mail.ru robert@mail.ru


2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [109]:
for i in json_selector(templates, 'phones'):
    for j in i:
        print(j['phone'])
    print()

232-19-55
+7 (916) 232-19-55

111-19-55
+7 (916) 445-19-55


3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [110]:
import json
with open ('C:/Users/Wiio/PycharmProjects/Makrushin_TOD/TOD_Homework_3/data/contributors_sample.json') as f:
    templates = json.load(f)
for i in range(3):
    print(templates[i])

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}
{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}


    1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [111]:
set((i[i.index('@'):] for i in json_selector(templates, 'mail')))

{'@gmail.com', '@hotmail.com', '@yahoo.com'}

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [112]:
def find_by_name(data, username: str) -> json:
    res=[]
    for i in data:
        if i['username'] == username:
            res.append(i)
    if len(res) == 0:
        raise ValueError('User not found')
    return res
print(find_by_name(templates, 'sheilaadams'))
find_by_name(templates, 'sheilaadam')

[{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}]


ValueError: User not found

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
al = json_selector(templates, 'sex')
print(f'Женщин: {al.count("F")}\nМужчин: {al.count("M")}')

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [114]:
from functools import partial
json_selector_2 = partial(json_selector, data=templates)

contributors = pd.DataFrame({'id':json_selector_2(param='id'),
                      'username':json_selector_2(param='username'),
                      'sex':json_selector_2(param='sex')})
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [124]:
a='C:/Users/Wiio/PycharmProjects/Makrushin_TOD/TOD_Homework_2/data/'
recipes = pd.read_csv(a + 'recipes_sample.csv', parse_dates=['submitted'], header=0)

merged = pd.merge(recipes, contributors, left_on='id', right_on='id', how='outer')  
merged['contributor_id'].isnull().sum()
merged


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,username,sex
0,boston cream pie,48,135.0,1545.0,1999-08-24,32.0,NaN,15.0,NaN,NaN
1,betty crocker s southwestern guacamole dip,55,125.0,1538.0,1999-09-15,NaN,NaN,5.0,NaN,NaN
2,black coffee barbecue sauce,66,30.0,174711.0,1999-09-10,NaN,it's great to know folks like this sauce so mu...,11.0,NaN,NaN
3,brown rice and vegetable pilaf,91,150.0,1576.0,1999-09-06,NaN,this is good with almost anything... robb,NaN,NaN,NaN
4,blueberry buttertarts,94,40.0,1556.0,1999-09-12,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
34025,NaN,2001662740,NaN,NaN,NaT,NaN,NaN,NaN,ejackson,M
34026,NaN,2001896867,NaN,NaN,NaT,NaN,NaN,NaN,fcline,F
34027,NaN,2001968497,NaN,NaN,NaT,NaN,NaN,NaN,ugutierrez,M
34028,NaN,2002059754,NaN,NaN,NaT,NaN,NaN,NaN,anthony87,M


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:

jobs = set()
employees = {}

for i in json_selector(templates, 'jobs'):
    for j in i:
        jobs.add(j)
        
jobs = list(jobs)

for i in templates:
    for j in i['jobs']:
        if j in jobs:
            employees.setdefault(j, []).append(i['username'])
            break

employees

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
with open('job_people.json', 'w') as f:     #118 КБ
    json.dump(employees, f, indent=4)
    
with open('job_people.pickle', 'wb') as f:  #70 КБ
    pickle.dump(employees, f)    
    

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.pickle', 'rb') as f:
    data = pickle.load(f)
data

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [125]:
with open('C:/Users/Wiio/PycharmProjects/Makrushin_TOD/TOD_Homework_3/data/low_steps.xml') as f:
    dom_tree = BeautifulSoup(f, 'xml')

ids = {}
# dom_tree.recipes.steps.getText('|', strip=True)
# d = {dom_tree.recipes.id.text}
# a = dom_tree.recipes.find('id')
# b = a.next_sibling.next_sibling.getText('|', strip=True)
# a.next
for id_tag in dom_tree.find_all("id"):
    id_text = id_tag.get_text(strip=True)
    
    steps = []
    steps_counter=0
    
    next_sibling = id_tag.next_sibling.next_sibling

    for i in next_sibling:
        if i.text == '\n':
            continue
        steps_counter+=1
        steps.append(f'Шаг {steps_counter}| {i.text}')

    
    ids[id_text] = steps  
    steps_counter=0
    ids[id_text] = steps
    steps = []

ids

{'44123': ['Шаг 1| in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes',
  'Шаг 2| add thyme , oregano and basil',
  'Шаг 3| saute 5 minutes more',
  'Шаг 4| add wine and deglaze pan',
  'Шаг 5| add hot chicken stock and reduce by one-third',
  'Шаг 6| add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets',
  'Шаг 7| simmer 5 minutes',
  'Шаг 8| add cream , simmer 5 minutes more and season to taste',
  'Шаг 9| drop in remaining butter , piece by piece , stirring until melted and serve immediately',
  'Шаг 10| smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare',
  'Шаг 11| chef meskan uses applewood chips and does not allow the grill to become too hot'],
 '67664': ['Шаг 1| mix all the ingredients using a blender',
  'Шаг 2| pour into popsicle molds',
  'Шаг 3| freeze and enjoy !'],
 '38798': ['Шаг 1| combine all ingredients in a large bowl and mix well',
  'Шаг 2| shape into one-inch 

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`